In [ ]:
%matplotlib inline
import os
from shutil import copyfile

import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import useful_rid_code as rid
from openpyxl import load_workbook

sn.set_context("notebook")

# RID 

## Fill-in OSPAR flows worksheet (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

In [ ]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

## 1. Discharges from monitored locations

The code below extracts summary statistics for 10 of the RID_11 stations from 1990 to 2016 inclusive.

In [ ]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# Read site data for RID_11 and RID_36
in_xlsx = r"../../../Data/RID_Sites_List.xlsx"
rid_11_df = pd.read_excel(in_xlsx, sheet_name="RID_11")
rid_36_df = pd.read_excel(in_xlsx, sheet_name="RID_36")

# Get just Suldalslågen from rid_36
rid_36_df = rid_36_df.query("station_id == 29781")

# Combine
mon_df = pd.concat([rid_11_df, rid_36_df], axis=0)

# Get OSPAR region for stations
sql = "SELECT station_id, value FROM resa2.stations_par_values WHERE var_id = 262"
ospar_reg = pd.read_sql_query(sql, engine)
ospar_reg.columns = ["station_id", "ospar_region"]

# Join OSPAR regions to station data
mon_df = pd.merge(mon_df, ospar_reg, how="left", on="station_id")

# Get cols of interest
mon_df["ospar_region"] = mon_df["ospar_region_x"]
mon_df = mon_df[["station_id", "station_code", "station_name", "ospar_region"]]

mon_df

In [ ]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# Container for results
df_list = []

# Loop over sites
for stn_id in mon_df["station_id"]:
    # Get catch area for chem station
    sql = f"SELECT catchment_area FROM resa2.stations WHERE station_id = {stn_id}"
    area_df = pd.read_sql_query(sql, engine)
    wc_area = area_df["catchment_area"].iloc[0]

    # Get linked discharge station
    sql = f"SELECT * FROM resa2.default_dis_stations WHERE station_id = {stn_id}"
    dis_df = pd.read_sql_query(sql, engine)
    dis_stn_id = dis_df["dis_station_id"].iloc[0]

    # Get catchment area for discharge station
    sql = (
        "SELECT area FROM resa2.discharge_stations "
        "WHERE dis_station_id = %s" % dis_stn_id
    )
    area_df = pd.read_sql_query(sql, engine)
    dis_area = area_df["area"].iloc[0]

    # Get annual summary flow stats for this station
    sql = (
        "SELECT TO_CHAR(xdate, 'YYYY') as year, "
        "       AVG(xvalue) as mean, "
        "       MIN(xvalue) as min, "
        "       MAX(xvalue) as max "
        "FROM resa2.discharge_values "
        "WHERE dis_station_id = %s "
        "AND xdate >= date '1990-01-01' "
        "AND xdate <= date '%s-12-31' "
        "GROUP BY TO_CHAR(xdate, 'YYYY') "
        "ORDER BY year" % (dis_stn_id, year)
    )
    q_df = pd.read_sql_query(sql, engine)

    # Set index
    q_df.index = q_df["year"]
    del q_df["year"]

    # Scale flows by area ratio
    q_df = q_df * wc_area / dis_area

    # Convert m3/s to 1000 m3/d
    q_df = q_df * 60 * 60 * 24 / 1000

    # Reset index
    q_df.reset_index(inplace=True)

    # Add LTA and n_yrs
    q_df["lta"] = q_df["mean"].mean()
    q_df["n_yrs"] = len(q_df)

    # Add ospar region ID, n_sites and 'mean' cols
    q_df["area_id"] = stn_id
    q_df["stat"] = "Mean"
    q_df["n_sites"] = 1

    # Re-order cols to match template
    q_df = q_df[
        ["area_id", "year", "mean", "lta", "min", "max", "n_yrs", "n_sites", "stat"]
    ]

    # Add to results
    df_list.append(q_df)

# Combine to single df
q_mon_df = pd.concat(df_list, axis=0)

q_mon_df.head()

## 2. Modelled

In [ ]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# Read cols of interest from Regine.txt
in_txt = r"../../../Recalculate_OSPAR_Flows/Regine.txt"
teo_df = pd.read_csv(in_txt, sep=";", comment="!", usecols=["Regine", "VASSOMR"])
teo_df.columns = ["regine", "vassdrag"]

# Read data from RESA2.RID_REGINE_DOWN
sql = "SELECT regine, area_fp FROM resa2.rid_regine_down"
ospar_reg = pd.read_sql_query(sql, engine)

# Join
nve_df = pd.merge(ospar_reg, teo_df, how="left", on="regine")

# Get cols of interest and simplify
nve_df = nve_df[["area_fp", "vassdrag"]]
nve_df.drop_duplicates(inplace=True)

## We already have monitored data for 10 of these vassdrags (see above)
## Remove these from consideration to avoid "double-counting"
## Get discharge station ids
# sql = ("SELECT * FROM resa2.default_dis_stations "
#       "WHERE station_id IN %s" % str(tuple(mon_df['station_id'].values)))
# dis_df = pd.read_sql_query(sql, engine)
#
## Get vassdrag numbers for montiored stations
# sql = ("SELECT nve_serienummer FROM resa2.discharge_stations "
#       "WHERE dis_station_id IN %s" % str(tuple(dis_df['dis_station_id'].values)))
# vass_nr = pd.read_sql_query(sql, engine)['nve_serienummer'].values
# vass_nr = [int(i.split('.')[0]) for i in vass_nr]
#
## Remove these vass_nrs from consideration in the modelled data
# nve_df = nve_df[~nve_df['vassdrag'].isin(vass_nr)]

nve_df.head()

In [ ]:
def combine_flow_data(vass, reg):
    """Function to combine time series for list of Vassdrags
        based on the NVE modelled data.

    Args:
        vass: List of strings. Vassdrags to combine
        reg:  Str. ID for region

    Returns:
        Dataframe. Annual summary stats calculated from the
        combined series (for 1990 to 2016)
    """
    # Get RESA2 station ID from vassdrag numbers
    sql = (
        "SELECT dis_station_id "
        "FROM resa2.discharge_stations "
        "WHERE nve_serienummer IN %s" % str(tuple(vass))
    )
    dis_ids = pd.read_sql_query(sql, engine)

    assert len(dis_ids) == len(vass), 'Lengths of "vass" and "dis_ids" do not match.'

    # Sum flow data for all sites in OSPAR reg to create a single
    # aggregated series
    # Get annual summary flow stats for this region
    sql = (
        "SELECT TO_CHAR(xdate, 'YYYY') as year, "
        "       AVG(xvalue) as mean, "
        "       MIN(xvalue) as min, "
        "       MAX(xvalue) as max "
        "FROM ( "
        "  SELECT TRUNC(xdate) AS xdate, "
        "         SUM(xvalue) AS xvalue "
        "  FROM resa2.discharge_values "
        "  WHERE dis_station_id IN %s "
        "  AND xdate >= DATE '1990-01-01' "
        "  AND xdate <= DATE '%s-12-31' "
        "  GROUP BY TRUNC(xdate) "
        "  ORDER BY TRUNC(xdate)) "
        "WHERE xdate >= date '1990-01-01' "
        "AND xdate <= date '%s-12-31' "
        "GROUP BY TO_CHAR(xdate, 'YYYY') "
        "ORDER BY year" % (str(tuple(dis_ids["dis_station_id"].values)), year, year)
    )
    q_df = pd.read_sql_query(sql, engine)

    # Set index
    q_df.index = q_df["year"]
    del q_df["year"]

    # Convert m3/s to 1000 m3/d
    q_df = q_df * 60 * 60 * 24 / 1000

    # Reset index
    q_df.reset_index(inplace=True)

    # Add LTA and n_yrs
    q_df["lta"] = q_df["mean"].mean()
    q_df["n_yrs"] = len(q_df)

    # Add ospar region ID, n_sites and 'mean' cols
    q_df["area_id"] = reg
    q_df["stat"] = "Mean"
    q_df["n_sites"] = len(vass)

    # Re-order cols to match template
    q_df = q_df[
        ["area_id", "year", "mean", "lta", "min", "max", "n_yrs", "n_sites", "stat"]
    ]

    return q_df

In [ ]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# Container for results
df_list = []

# Loop over OSPAR areas
for osp_reg in nve_df["area_fp"].unique():
    # Get all vassdrags draining to this region
    vass = nve_df.query("area_fp == @osp_reg")["vassdrag"].values.astype(str)

    # Get stats
    q_df = combine_flow_data(vass, osp_reg)

    # Add to results
    df_list.append(q_df)

# Single calculation for the whole of norway
vass = nve_df["vassdrag"].values.astype(str)
q_df = combine_flow_data(vass, "all_nor")
df_list.append(q_df)

# Combine to single df
q_mod_df = pd.concat(df_list, axis=0)

q_mod_df.head()

## 3. Write results to template

The code below iterates over the output and writes the results to the Excel template.

In [ ]:
# Path to template
osp_path = f"../../../Results/OSPAR/01_OSPAR_Norway_{year}.xlsx"

# Dict mappiong names in template to area IDs in dfs
name_dict = {
    "Orkla": 29778,
    "Vefsna": 29782,
    "Norwegian Sea (NO)": "91_170",
    "Alta": 29779,
    "Barents Sea (NO)": "171_247",
    "Glomma": 29617,
    "Drammenselva": 29612,
    "Numedalslågen": 29615,
    "Skienselva": 29613,
    "Otra": 29614,
    "Inner Oslofjord": 36225,  # Assume just Alna for now(?)
    "Skagerrak (NO)": "1_23",
    "Orreelva": 29783,
    "Suldalslågen": 29781,
    "Vosso": 29821,
    "North Sea (NO)": "24_90",
    "Norway Total": "all_nor",
}

# Open new file and get sheet
wb = load_workbook(filename=osp_path)
ws = wb["9"]

year = str(year)
for item in ws["B12":"B28"]:
    # Get cell properties
    cell = item[0]
    area = cell.value
    row = cell.row

    # Get area ID
    ar_id = name_dict[area]

    if ar_id != 999:
        # Get data from relevant df
        if isinstance(ar_id, int):
            # Monitored df
            df = q_mon_df.query("(area_id == @ar_id) and (year == @year)")
        else:
            # Modelled df
            df = q_mod_df.query("(area_id == @ar_id) and (year == @year)")

        assert len(df) == 1

        # Write values
        # 1. Mean
        ws.cell(column=5, row=row, value=df.iloc[0]["mean"])

        # 2. LTA
        ws.cell(column=7, row=row, value=df.iloc[0]["lta"])

        # 3. Min
        ws.cell(column=9, row=row, value=df.iloc[0]["min"])

        # 4. Max
        ws.cell(column=11, row=row, value=df.iloc[0]["max"])

        # 5. Years
        ws.cell(column=13, row=row, value=df.iloc[0]["n_yrs"])

        # 6. N_Sites
        ws.cell(column=15, row=row, value=df.iloc[0]["n_sites"])

        # 7. Stat
        ws.cell(column=17, row=row, value=df.iloc[0]["stat"])

# Save
wb.save(osp_path)